In [108]:
# -*- coding: utf-8 -*-

Zaimplementuj aplikację szacującą czas ukończenia półmaratonu dla zadanych danych

1. Umieść dane w Digital Ocean Spaces

1. Napisz notebook, który będzie Twoim pipelinem do trenowania modelu
    * czyta dane z Digital Ocean Spaces
    * czyści je
    * trenuje model (dobierz odpowiednie metryki [feature selection])
    * nowa wersja modelu jest zapisywana lokalnie i do Digital Ocean Spaces

1. Aplikacja
    * opakuj model w aplikację streamlit
    * wdróż (deploy) aplikację za pomocą Digital Ocean AppPlatform 
    * wejściem jest pole tekstowe, w którym użytkownik się przedstawia, mówi o tym
    jaka jest jego płeć, wiek i czas na 5km
    * jeśli użytkownik podał za mało danych, wyświetl informację o tym jakich danych brakuje
    * za pomocą LLM (OpenAI) wyłuskaj potrzebne dane, potrzebne dla Twojego modelu
    do określenia, do słownika (dictionary lub JSON)
    * tę część podepnij do Langfuse, aby zbierać metryki o skuteczności działania LLM'a



In [147]:
import sys
import io
import os
import getpass
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import csv
from faker import Faker
from pycaret.classification import setup, pull
import re
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import homogeneity_score, completeness_score,accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from pycaret.clustering import *
from dotenv import set_key, load_dotenv
from itables import show
import boto3


# Poprawiony import Pandera
import pandera as pa
from pandera import Column, DataFrameSchema, Check

# Import PyCaret
from pycaret.clustering import setup, create_model, save_model, plot_model, assign_model, predict_model, pull

# Ustawienie ścieżki
current_dir = Path.cwd() 
#print(f"Aktualny folder to: {current_dir}")




# Funkcje odpowiedzialne za komunikację z digital ocean i openai

Funkcje próbują wczytać klucze z pliku env, a w przypadku braku pokazują pole tekstowe do wprowadzenia danych. Dalej mamy też funkcję wysyłającą dane na serwer digital ocean.

In [148]:
def handle_digital_ocean_keys():
    # 1. Próba załadowania z .env
    current_folder = Path().absolute()
    env_path = current_folder / ".env"
    load_dotenv(env_path)
    #print(env_path)
    # Pobranie wartości z systemu/pliku .env
    do_access_key = os.getenv("DO_ACCESS_KEY")
    do_secret_key = os.getenv("DO_SECRET_KEY")
    do_space_input = os.getenv("DO_SPACE_NAME")
    do_region = os.getenv("DO_REGION")
    do_end_url = os.getenv("DO_END_URL")
    
    # Sprawdzamy, czy klucze już są załadowane
    keys_loaded = all([do_access_key, do_secret_key, do_space_input])

    if not keys_loaded:
        print("Konfiguracja DigitalOcean Spaces")
        do_access_key = getpass.getpass("Wprowadź DigitalOcean Access Key: ")
        do_secret_key = getpass.getpass("Wprowadź secret digital oceans space Key")
        do_space_input = input("Wprowadź nazwę Space (Bucketa): ")
        do_region = input("Wprowadź region (np. fra1): ")
        do_end_url = input("Wprowadź url region (np. https://fra1.digitaloceanspaces.com): ")
                
        if do_access_key and do_secret_key and do_space_input:
            # Ustawienie zmiennych środowiskowych, aby os.getenv je widział po rerun
            os.environ["DO_ACCESS_KEY"] = do_access_key
            os.environ["DO_SECRET_KEY"] = do_secret_key
            os.environ["DO_SPACE_NAME"] = do_space_input
            os.environ["DO_REGION"] = do_region
            os.environ["DO_END_URL"] = do_end_url
            set_key(str(env_path), "DO_ACCESS_KEY", do_access_key)
            set_key(str(env_path), "DO_SECRET_KEY", do_secret_key)
            set_key(str(env_path), "DO_SPACE_NAME", do_space_input)
            set_key(str(env_path), "DO_REGION", do_region)
            set_key(str(env_path), "DO_END_URL", do_end_url)
            print("Klucze zapisane!")
        else:
            print("Wypełnij wszystkie pola!")
            
    return do_access_key, do_secret_key, do_space_input, do_region, do_end_url

In [149]:
def digital_ocean_to_csv(access_key,access_secret_key, nazwa_space,plik_csv,region = 'fra1',end_url='https://fra1.digitaloceanspaces.com'):
    load_dotenv()

    # Konfiguracja sesji
    session = boto3.session.Session()
    client = session.client(
        's3',
        region_name=region.strip(),
        endpoint_url=end_url.strip(), # Używamy endpointu bazowego do komunikacji API
        aws_access_key_id=access_key.strip(),
        aws_secret_access_key=access_secret_key.strip()
    )

    # Pobieranie pliku bezpośrednio do pamięci (bez zapisu na dysku)
    response = client.get_object(Bucket=nazwa_space, Key=plik_csv)
    csv_data = response['Body'].read()

    # Wczytanie do Pandas
    return pd.read_csv(io.BytesIO(csv_data), sep=';',encoding='utf-8-sig')

In [150]:
def upload_model_to_digital_ocean(local_file_path, access_key, access_secret_key, nazwa_space, remote_path=None, region='fra1', end_url='https://fra1.digitaloceanspaces.com'):
    # Jeśli nie podano nazwy w chmurze, użyj nazwy pliku lokalnego
    if remote_path is None:
        remote_path = os.path.basename(local_file_path)
        
    session = boto3.session.Session()
    client = session.client(
        's3',
        region_name=region.strip(),
        endpoint_url=end_url.strip(),
        aws_access_key_id=access_key.strip(),
        aws_secret_access_key=access_secret_key.strip()
    )

    try:
        # Wysyłanie pliku binarnego .pkl
        client.upload_file(local_file_path, nazwa_space, remote_path)
        print(f"Model {local_file_path} został wysłany do Space: {nazwa_space} jako {remote_path}")
    except Exception as e:
        print(f"Błąd wysyłki modelu: {e}")


# Funkcje pomocnicze

Funkcje pomocnicze odpowiedzialne za przeprocesowanie datasetu do uczenia maszynowego oraz do konwersji czasu z formatu time na ilość sekund

In [151]:
# Snippet pomocniczy - zmiana czasu na sekundy

def convert_time_to_seconds(time):
      # Jeśli wartość to NaN (float) lub nie jest stringiem, zwróć None lub 0
    if not isinstance(time, str):
        return 0  # lub return 0
    if pd.isnull(time) or time in ['DNS', 'DNF']:
        return 0
    time = time.split(':')
    return int(time[0]) * 3600 + int(time[1]) * 60 + int(time[2])

In [152]:
def bezpieczna_dominanta(seria, wartosc_domyslna):
    m = seria.mode()
    return m.iloc[0] if not m.empty else wartosc_domyslna

# Wczytanie danych z digital ocean

Wczytujemy wcześniej zapisane dane z digital ocean oraz wyświetlamy ilość wczytanych wierszy, by mieć pewność poprawności wczytania.

In [153]:
#df_maraton23 = pd.read_csv('halfmarathon_wroclaw_2023__final.csv', encoding='utf-8-sig',sep=';')
do_access_key, do_secret_key, do_space_input, do_region, do_end_url = handle_digital_ocean_keys()
df_maraton23 = digital_ocean_to_csv(do_access_key, do_secret_key, do_space_input, 'halfmarathon_wroclaw_2023__final.csv',do_region, do_end_url)
df_maraton23.columns = df_maraton23.columns.str.strip()
print(f"Pomyślnie wczytano dane. Liczba wierszy: {len(df_maraton23)}")

Pomyślnie wczytano dane. Liczba wierszy: 8950


In [154]:
#df_maraton24 = pd.read_csv('halfmarathon_wroclaw_2024__final.csv', encoding='utf-8-sig', sep=';')
do_access_key, do_secret_key, do_space_input, do_region, do_end_url = handle_digital_ocean_keys()
df_maraton24 = digital_ocean_to_csv(do_access_key, do_secret_key, do_space_input, 'halfmarathon_wroclaw_2024__final.csv',do_region, do_end_url)
df_maraton24.columns = df_maraton23.columns.str.strip()
print(f"Pomyślnie wczytano dane. Liczba wierszy: {len(df_maraton24)}")

Pomyślnie wczytano dane. Liczba wierszy: 13007


# Wypełanianie brakujących danych

Rzutujemy poszczególne kolumny na dany typ i wypełniamy brakujące dane. Kolumny liczbowe wypełniamy zerami. Poszczególne odcinki czasowe zamieniamy z formatu time na ilość sekund w celu ułatwienia dalszej analizy
 

In [155]:
df_maraton23['Miejsce'] = pd.to_numeric(df_maraton23['Miejsce'], errors='coerce').fillna(0).astype('Int64')
df_maraton23['Płeć Miejsce'] = pd.to_numeric(df_maraton23['Płeć Miejsce'], errors='coerce').fillna(0).astype('Int64')
df_maraton23['Kategoria wiekowa Miejsce'] = pd.to_numeric(df_maraton23['Kategoria wiekowa Miejsce'], errors='coerce').fillna(0).astype('Int64')
df_maraton23['Rocznik'] = pd.to_numeric(df_maraton23['Rocznik'], errors='coerce').fillna(0).astype('Int64')
df_maraton23['Czas'] = df_maraton23['Czas'].apply(convert_time_to_seconds)
df_maraton23['5 km Czas'] = df_maraton23['5 km Czas'].apply(convert_time_to_seconds)
df_maraton23['10 km Czas'] = df_maraton23['10 km Czas'].apply(convert_time_to_seconds)
df_maraton23['15 km Czas'] = df_maraton23['15 km Czas'].apply(convert_time_to_seconds) 
df_maraton23['20 km Czas'] = df_maraton23['20 km Czas'].apply(convert_time_to_seconds) 


In [156]:
df_maraton23.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
8884,0,2943,PAWEŁ,WYSOCZAŃSKI,NaN,NaN,Tri Granda Wołów,M,0,M30,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
2771,2772,6728,TOMASZ,ŻŁOBIŃSKI,TYCHY,POL,NaN,M,2460,M30,...,5.113333,4653,2655.0,5.430000,6436,2724.0,5.943333,0.064933,6777,5.354349
4345,4346,4095,KRZYSZTOF,NOWAK,OWCZEGŁOWY,POL,ATLAS RUNNING TEAM,M,3623,M40,...,5.366667,4926,3786.0,5.843333,6949,4339.0,6.743333,0.101533,7291,5.760449
5817,5818,2094,ŁUKASZ,KRUKOWSKI,ŁOWICZ,POL,NaN,M,4597,M30,...,5.870000,5445,5810.0,6.263333,7537,5782.0,6.973333,0.065267,7927,6.262938
749,750,2680,BARTOSZ,PŁONKA,WROCŁAW,POL,NaN,M,700,M40,...,4.243333,3942,550.0,4.610000,5592,733.0,5.500000,0.080133,5882,4.647231
6636,6637,8908,JAKUB,GRODZICKI,WROCŁAW,POL,NaN,M,5055,M20,...,6.530000,5985,7069.0,6.710000,8032,6626.0,6.823333,0.010400,8440,6.668247
7584,7585,2429,JAN,JARUZEL,TOMNICE,POL,KS KROTOSZ KROTOSZYN,M,5534,M50,...,7.306667,6185,7379.0,7.136667,8860,7595.0,8.916667,0.161200,9305,7.351663
6889,6890,4185,MARTA,KOŁODZIEJ,NOWA SÓL,POL,NaN,K,1702,K40,...,6.543333,5922,6936.0,6.666667,8227,6899.0,7.683333,0.071667,8643,6.828632
6366,6367,4376,ALEKSANDRA,ZIELIŃSKA,KRAKÓW,POL,NaN,K,1452,K30,...,6.553333,5774,6631.0,6.353333,7869,6406.0,6.983333,0.034600,8235,6.506281
1399,1400,6358,MARCIN,MYŚLIWEK,TRZEBNICA,POL,16DBOT,M,1294,M40,...,4.710000,4347,1449.0,4.976667,5951,1405.0,5.346667,0.037933,6238,4.928498


In [157]:
df_maraton24['Miejsce'] = pd.to_numeric(df_maraton24['Miejsce'], errors='coerce').fillna(0).astype('int64')
df_maraton24['Płeć Miejsce'] = pd.to_numeric(df_maraton24['Płeć Miejsce'], errors='coerce').fillna(0).astype('int64')
df_maraton24['Kategoria wiekowa Miejsce'] = pd.to_numeric(df_maraton24['Kategoria wiekowa Miejsce'], errors='coerce').fillna(0).astype('int64')
df_maraton24['Rocznik'] = pd.to_numeric(df_maraton24['Rocznik'], errors='coerce').fillna(0).astype('int64')
df_maraton24['Czas'] = df_maraton24['Czas'].apply(convert_time_to_seconds)
df_maraton24['5 km Czas'] = df_maraton24['5 km Czas'].apply(convert_time_to_seconds)
df_maraton24['10 km Czas'] = df_maraton24['10 km Czas'].apply(convert_time_to_seconds)
df_maraton24['15 km Czas'] = df_maraton24['15 km Czas'].apply(convert_time_to_seconds) 
df_maraton24['20 km Czas'] = df_maraton24['20 km Czas'].apply(convert_time_to_seconds) 

In [158]:
df_maraton24.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
1622,1623,7009,MARIUSZ,NIEDWOROK,OTMICE,POL,IZBICKO W BIEGU,M,1498,M40,...,4.753333,4411,1780.0,4.956667,5955,1701.0,5.146667,0.013267,6276,4.958521
5281,5282,66,KATARZYNA,PLEWA,STRONIE ŚLASKIE,POL,GÓRYLASY,K,875,K40,...,5.456667,5020,4720.0,5.986667,6923,5232.0,6.343333,0.073800,7352,5.808643
11731,0,6337,ANONIMOWY,ZAWODNIK,NaN,NaN,NaN,M,0,M30,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
6545,6546,7615,BARTOSZ,POSIŁEK,GUBIN,POL,NaN,M,5245,M30,...,5.673333,5326,6316.0,6.283333,7308,6507.0,6.606667,0.060800,7766,6.135735
6881,6882,6963,PIOTR,LEPCZYŃSKI,POZNAN,POL,BRAK,M,5467,M30,...,5.753333,5341,6393.0,6.246667,7393,6773.0,6.840000,0.072067,7871,6.218693
10535,0,79534,ACHIN,CHATURVEDI,NaN,NaN,Swords Athletics,M,0,M20,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,0,NaN
376,377,347,KAMIL,IWAN,HUTA NOWA,POL,DREAM TEAM WOKÓŁ ŁYSEJ GÓRY,M,358,M30,...,4.040000,3785,313.0,4.406667,5182,366.0,4.656667,0.036533,5498,4.343841
1038,1039,1562,TOMASZ,BĄK,JELCZ-LASKOWICE,POL,TOYOTA RUNMATOR,M,981,M40,...,4.453333,4145,936.0,4.720000,5643,997.0,4.993333,0.026333,5988,4.730979
5830,5831,8923,MICHAŁ,WYLĘŻEK,KIEŁCZÓW,POL,NaN,M,4789,M30,...,5.663333,5251,5973.0,5.986667,7107,5816.0,6.186667,0.026467,7536,5.954018
5715,5716,5700,STANISŁAW,SKÓRSKI,SZCZECIN,POL,RAZ SZCZECIN,M,4713,M70,...,5.696667,5258,6000.0,6.050000,7099,5787.0,6.136667,0.028467,7506,5.930315


# Wypełniamy brakujące wartości w danych demograficznych

Braki w danych demograficznych zostały uzupełnione wartościami typowymi dla tego zbioru (dominanta). Dzięki temu model klastrowania może przetworzyć wszystkie rekordy, zachowując spójność z profilem typowego uczestnika maratonu

In [159]:
df_maraton23['Płeć'] = df_maraton23['Płeć'].fillna(bezpieczna_dominanta(df_maraton23['Płeć'], 'K'))
df_maraton23['Kategoria wiekowa'] = df_maraton23['Kategoria wiekowa'].fillna(bezpieczna_dominanta(df_maraton23['Kategoria wiekowa'], 'M0')) 
df_maraton23['Rocznik'] = df_maraton23['Rocznik'].fillna(bezpieczna_dominanta(df_maraton23['Rocznik'], '1970'))

In [160]:
df_maraton24['Płeć'] = df_maraton24['Płeć'].fillna(bezpieczna_dominanta(df_maraton24['Płeć'], 'K'))
df_maraton24['Kategoria wiekowa'] = df_maraton24['Kategoria wiekowa'].fillna(bezpieczna_dominanta(df_maraton24['Kategoria wiekowa'], 'M0')) 
df_maraton24['Rocznik'] = df_maraton24['Rocznik'].fillna(bezpieczna_dominanta(df_maraton24['Rocznik'], '1970'))

# Wypełnianie brakujących wartości w statystykach biegowych

W celu uzupełnienia braków w międzyczasach zastosowano interpolację liniową. Wybór tej metody podyktowany jest charakterystyką danych procesowych (bieg ciągły) – pozwala ona na oszacowanie brakujących wartości w oparciu o dynamikę tempa konkretnego zawodnika pomiędzy najbliższymi znanymi punktami pomiarowymi, co minimalizuje ryzyko wprowadzenia błędów do analizy stabilności biegu.

In [161]:
# Lista kolumn do interpolacji
kolumny_czasowe = ['Czas', '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas','5 km Tempo','10 km Tempo','15 km Tempo','20 km Tempo','Tempo','Tempo Stabilność']

# 1. Przygotowanie danych i zamiana zer na NaN
temp_df = df_maraton23.copy()
for col in kolumny_czasowe:
    temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce').replace(0, np.nan)

# 2. Sortowanie
temp_df = temp_df.sort_values(by=['Płeć', 'Kategoria wiekowa', 'Numer startowy'])

# 3. Interpolacja CAŁEGO DataFrame, a następnie wypełnianie grupowe (bezpieczna kolejność)
# Interpolacja globalna (ominięcie problemu grup)
temp_df[kolumny_czasowe] = temp_df[kolumny_czasowe].interpolate(method='linear', limit_direction='both')

# 4. Wypełnianie pozostałych braków średnią grupową
# Użycie transform jest bezpieczne, bo fillna nie generuje błędów metody
temp_df[kolumny_czasowe] = temp_df.groupby(['Płeć', 'Kategoria wiekowa'])[kolumny_czasowe].transform(
    lambda x: x.fillna(x.mean())
)

# 5. Finalne wypełnienie
df_maraton23 = temp_df
df_maraton23 = df_maraton23.fillna('Brak') 
df_maraton23.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
743,744,7356,PIOTR,KINAL,KŁODZKO,POL,KLUB LEKKOATLETYCZNY ZIEMIA KŁODZKA,M,694,M40,...,4.243333,4172.0,949.0,5.890000,5642.0,811.0,4.900000,0.100533,5880.0,4.645651
7398,7399,7444,ANNA,BUCHOWIECKA,SULEJÓWEK,POL,PSTROKATY PAREOMARATON,K,1948,K40,...,6.663333,6032.0,7163.0,7.170000,8659.0,7397.0,8.756667,0.159133,9100.0,7.189697
5102,5103,4385,SERGIUSZ,SZULGAN,WROCŁAW,POL,SZALONY DZIK,M,4143,M30,...,5.773333,5257.0,5217.0,6.113333,7225.0,5088.0,6.560000,0.062200,7589.0,5.995892
4228,4229,1358,DANIEL,WABNIC,KSIĘGINICE,POL,Brak,M,3539,M40,...,5.520000,5042.0,4327.0,5.603333,6899.0,4189.0,6.190000,0.032067,7251.0,5.728846
1831,1832,8202,JAN,RYBA,WROCŁAW,POL,BIEGAM BO LUBIĘ WROCŁAW,M,1674,M60,...,4.786667,4450.0,1835.0,5.323333,6115.0,1825.0,5.550000,0.060333,6422.0,5.073872
7143,7144,2828,GRZEGORZ,LICHOSIK,WROCŁAW,POL,SSSSSY,M,5313,M40,...,6.770000,5918.0,6928.0,6.610000,8459.0,7176.0,8.470000,0.124200,8851.0,6.992968
3959,3960,3621,ARTUR,GOS,VILLINGEN,GER,Brak,M,3345,M20,...,5.466667,5011.0,4180.0,5.726667,6845.0,3981.0,6.113333,0.041400,7167.0,5.662479
6590,6591,1545,JAKUB,SIKORA,WILKSZYN,POL,Brak,M,5034,M40,...,6.360000,5782.0,6647.0,6.763333,8006.0,6586.0,7.413333,0.083867,8404.0,6.639804
8196,0,5476,MAŁGORZATA,BOLEK-HULACKA,Brak,Brak,Brak,K,0,K40,...,6.805000,6253.5,Brak,7.380000,8913.0,Brak,8.865000,0.143800,9410.0,7.434621
1364,1365,2615,MARTA,SONNEK,SYCÓW,POL,Brak,K,104,K40,...,4.720000,4311.0,1335.0,4.936667,5925.0,1338.0,5.380000,0.044333,6226.0,4.919017


In [162]:
# Lista kolumn do interpolacji
kolumny_czasowe = ['Czas', '5 km Czas', '10 km Czas', '15 km Czas', '20 km Czas','5 km Tempo','10 km Tempo','15 km Tempo','20 km Tempo','Tempo','Tempo Stabilność']

# 1. Przygotowanie danych i zamiana zer na NaN
temp_df = df_maraton24.copy()
for col in kolumny_czasowe:
    temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce').replace(0, np.nan)

# 2. Sortowanie
temp_df = temp_df.sort_values(by=['Płeć', 'Kategoria wiekowa', 'Numer startowy'])

# 3. Interpolacja CAŁEGO DataFrame, a następnie wypełnianie grupowe (bezpieczna kolejność)
# Interpolacja globalna (ominięcie problemu grup)
temp_df[kolumny_czasowe] = temp_df[kolumny_czasowe].interpolate(method='linear', limit_direction='both')

# 4. Wypełnianie pozostałych braków średnią grupową
# Użycie transform jest bezpieczne, bo fillna nie generuje błędów metody
temp_df[kolumny_czasowe] = temp_df.groupby(['Płeć', 'Kategoria wiekowa'])[kolumny_czasowe].transform(
    lambda x: x.fillna(x.mean())
)


# 5. Finalne wypełnienie
df_maraton24 = temp_df
df_maraton24 = df_maraton24.fillna('Brak') 
df_maraton24.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
12677,0,86181,RAFAŁ,WAWRZKOWICZ,Brak,Brak,Biegowe Wtorki,M,0,M40,...,4.716009,4379.901408,Brak,5.077981,6017.239437,Brak,5.457793,0.046366,6407.084507,5.062088
11609,0,76750,MATEUSZ,MAJSIAK,Brak,Brak,Warta Osjaków,M,0,M20,...,5.555455,5110.696970,Brak,5.896263,6975.727273,Brak,6.216768,0.044786,7396.575758,5.843862
3066,3067,1709,DARIUSZ,NOWAKOWSKI,Brak,POL,Brak,M,2730,M20,...,5.120000,4613.000000,2659.0,5.386667,6348.000000,2937.0,5.783333,0.060133,6767.000000,5.346449
4555,4556,8526,BARTOSZ,BIELECKI,WROCŁAW,POL,Brak,M,3892,M30,...,5.460000,4965.000000,4386.0,5.700000,6758.000000,4543.0,5.976667,0.040000,7159.000000,5.656159
8245,8246,10740,NATALIA,WAWRZYNIEWICZ,BORGLOON,BEL,Brak,K,2037,K40,...,6.473333,5967.000000,8659.0,6.793333,7999.000000,8330.0,6.773333,0.015400,8444.000000,6.671407
1762,1763,7545,BOGUSLAW,MAJDA,NYSA,POL,ZWARIOWANY BODZIO,M,1623,M50,...,4.730000,4401.000000,1742.0,4.913333,5958.000000,1718.0,5.190000,0.013467,6318.000000,4.991704
8508,8509,10997,MAŁGORZATA,JAROSIK,LESZNO,POL,Brak,K,2173,K40,...,6.486667,6014.000000,8770.0,6.516667,8104.000000,8531.0,6.966667,-0.004000,8574.000000,6.774117
10081,10084,10445,MAGDALENA,FRĄCZYK,NADOLICE WIELKIE,POL,KB HARCOWNIK JELCZ LASKOWICE,K,2941,K40,...,7.763333,7118.000000,10145.0,8.553333,9712.000000,10104.0,8.646667,0.090000,10287.000000,8.127518
6365,6366,6751,MAŁGORZATA,RÓŻAŃSKA,WROCŁAW,POL,PRO-RUN WROCŁAW,K,1235,K50,...,5.596667,5209.000000,5799.0,6.103333,7236.000000,6271.0,6.756667,0.075733,7703.000000,6.085960
576,577,2430,ROBERT,MATKOWSKI,SOBÓTKA,POL,CARO TEAM,M,550,M40,...,4.196667,3919.000000,503.0,4.453333,5349.000000,554.0,4.766667,0.026333,5675.000000,4.483685


# Ranking i obsługa miejsc

Dla kolumn określających pozycję zawodników (Open, Płeć oraz Międzyczasy) zastosowano metodę rankingu 'min' (Competition Ranking). Wybór ten podyktowany jest chęcią zachowania spójności z oficjalnymi zasadami sędziowania zawodów sportowych, gdzie zawodnicy z identycznym czasem zajmują tę samą pozycję, a kolejny numer miejsca jest odpowiednio przesunięty.
W procesie modelowania cechy te zostały wyłączone z obliczeń (ignored features), aby uniknąć redundancji danych z kolumnami czasowymi, jednak zachowano je jako kluczowy kontekst dla modelu językowego (LLM) przy generowaniu opisów klastrów.

In [163]:
df_maraton23['Miejsce'] = df_maraton23['Czas'].rank(method='min').astype('Int64')
df_maraton23['Płeć Miejsce'] = df_maraton23.groupby('Płeć')['Czas'].rank(method='min').astype('Int64')
df_maraton23['5 km Miejsce Open'] = df_maraton23['5 km Czas'].rank(method='min').astype('Int64')
df_maraton23['10 km Miejsce Open'] = df_maraton23['10 km Czas'].rank(method='min').astype('Int64')
df_maraton23['15 km Miejsce Open'] = df_maraton23['15 km Czas'].rank(method='min').astype('Int64')
df_maraton23['20 km Miejsce Open'] = df_maraton23['20 km Czas'].rank(method='min').astype('Int64')

df_maraton24['Miejsce'] = df_maraton24['Czas'].rank(method='min').astype('Int64')
df_maraton24['Płeć Miejsce'] = df_maraton24.groupby('Płeć')['Czas'].rank(method='min').astype('Int64')
df_maraton24['5 km Miejsce Open'] = df_maraton24['5 km Czas'].rank(method='min').astype('Int64')
df_maraton24['10 km Miejsce Open'] = df_maraton24['10 km Czas'].rank(method='min').astype('Int64')
df_maraton24['15 km Miejsce Open'] = df_maraton24['15 km Czas'].rank(method='min').astype('Int64')
df_maraton24['20 km Miejsce Open'] = df_maraton24['20 km Czas'].rank(method='min').astype('Int64')


In [164]:
#kolumny = ['Imię', 'Nazwisko', 'Numer startowy', 'Kategoria wiekowa',"Czas"]
#df_maraton23.loc[df_maraton23['Kategoria wiekowa'].isna(), kolumny]
df_maraton24.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
2737,3069,2819,RADOSŁAW,PIETRZAK,MROWINY,POL,BRT,M,2758,M50,...,5.083333,4701.000000,3492,5.296667,6302.000000,3144,5.336667,0.007067,6657.000000,5.259540
12297,11472,25936,WIOLETA,SIKORSKA,Brak,Brak,Brak,K,3183,K40,...,6.764242,6173.009569,11519,7.330431,8439.397129,11475,7.554625,0.075680,8942.373206,7.065160
10426,10126,21652,WOJCIECH,BLASZCZYK,Brak,Brak,Atek_falko,M,7625,M30,...,6.420249,5898.385928,10663,6.693397,7928.049041,10272,6.765544,0.018537,8347.507463,6.595171
9756,12360,10446,TOMASZ,HAWRON,WROCŁAW,POL,PARKRUN WROCŁAW,M,8733,M50,...,7.366667,6728.000000,12546,7.846667,9087.000000,12368,7.863333,0.048600,9633.000000,7.610808
5416,6336,5107,DARIUSZ,KOWALKOWSKI,WROCŁAW,POL,MATNER RUNNING TEAM,M,5315,M40,...,5.703333,5249.000000,7079,5.906667,7028.000000,6538,5.930000,0.006667,7394.000000,5.841827
803,840,435,OLIMPIA,REZAI,WIEN,AUT,LG WIENVIENNA RUNNING COLLECTIVE,K,43,K40,...,4.416667,4062.000000,787,4.670000,5523.000000,815,4.870000,0.030067,5869.000000,4.636960
10849,3511,28796,KRYSTIAN,GAŁWA,Brak,Brak,Brak,M,3134,M40,...,5.091847,4689.610329,3432,5.255196,6379.755869,3445,5.633818,0.024197,6774.953052,5.352732
5387,6300,11810,AGNIESZKA,CHOJECKA,KĄTY WROCŁAWSKIE,POL,Brak,K,1012,K40,...,5.660000,5199.000000,6787,5.873333,6982.000000,6356,5.943333,0.013067,7383.000000,5.833136
8224,10379,11730,DAMIAN,ŁUKASZ,WROCŁAW,POL,WROCLAW MARATHON TEAM,M,7752,M40,...,5.776667,5490.000000,8573,6.573333,7875.000000,10081,7.950000,0.135933,8432.000000,6.661926
13006,7744,80025,KRZYSZTOF,ĆWIĘK,Brak,Brak,Pko,M,6188,M30,...,5.744627,5293.635394,7365,6.150142,7295.997868,7718,6.674542,0.063542,7745.238806,6.119332


# Walidacja Schematu Danych (Pandera)

Po zakończeniu procesu preprocessingu i inżynierii cech, dane zostają poddane rygorystycznej walidacji schematu przy użyciu biblioteki Pandera.
Cel tego kroku:

    Gwarancja spójności typów: Upewnienie się, że kolumny numeryczne (np. Tempo, Czas) są faktycznie liczbami, a kategoryczne (np. Płeć) zawierają tylko dozwolone wartości ("M", "K").
    Zabezpieczenie modelu ML: Model klastrujący oczekuje konkretnego zestawu wejściowego. Walidacja schematu zapobiega błędom w fazie predykcji, które mogłyby wyniknąć ze zmiany formatu danych wejściowych.
    Obsługa braków: Jawne zdefiniowanie parametrów nullable=True pozwala na kontrolowany nadzór nad rekordami, które posiadają niepełne informacje.

Jest to ostatnia barierka ochronna przed wdrożeniem aplikacji na produkcję w DigitalOcean, gwarantująca, że skrypt nie przerwie pracy z powodu niespodziewanej zmiany w strukturze plików CSV.

In [165]:
schema = pa.DataFrameSchema(
    columns={
        "Miejsce": pa.Column(pa.Int, nullable=True),
        "Numer startowy": pa.Column(pa.Int, nullable=True),
        "Imię": pa.Column(pa.String),
        "Nazwisko": pa.Column(pa.String),
        "Miasto": pa.Column(pa.String, nullable=True),
        "Kraj": pa.Column(pa.String, nullable=True),
        "Drużyna": pa.Column(pa.String, nullable=True),
        "Płeć": pa.Column(pa.String, pa.Check.isin(["M", "K"])),
        "Płeć Miejsce": pa.Column(pa.Int, nullable=True),
        "Kategoria wiekowa": pa.Column(pa.String),
        "Kategoria wiekowa Miejsce": pa.Column(pa.Int, nullable=True),
        "Rocznik": pa.Column(pa.Int, nullable=True),
        "5 km Czas": pa.Column(pa.Float),
        "5 km Miejsce Open": pa.Column(pa.Int, nullable=True),
        "5 km Tempo": pa.Column(pa.Float),
        "10 km Czas": pa.Column(pa.Float),
        "10 km Miejsce Open": pa.Column(pa.Int, nullable=True),
        "10 km Tempo": pa.Column(pa.Float),
        "15 km Czas": pa.Column(pa.Float),
        "15 km Miejsce Open": pa.Column(pa.Int, nullable=True),
        "15 km Tempo": pa.Column(pa.Float),
        "20 km Czas": pa.Column(pa.Float),
        "20 km Miejsce Open": pa.Column(pa.Int, nullable=True),
        "20 km Tempo": pa.Column(pa.Float),
        "Tempo Stabilność": pa.Column(pa.Float, nullable=True),
        "Czas": pa.Column(pa.Float),
        "Tempo": pa.Column(pa.Float, nullable=True),
    },
    strict=True, # Sprawdza, czy nie ma nadmiarowych kolumn
    coerce=True  # Automatycznie próbuje naprawić typy (np. str na int)
)

In [166]:
try:
    schema.validate(df_maraton23)
    print("Dane maraton23 są poprawne")
except pa.errors.SchemaError as e:
    print(f"Błąd walidacji: {e}")

Dane maraton23 są poprawne


In [167]:
try:
    schema.validate(df_maraton24)
    print("Dane maraton24 są poprawne")
except pa.errors.SchemaError as e:
    print(f"Błąd walidacji: {e}")


Dane maraton24 są poprawne


# Inżynieria Cech i Preprocessing Końcowy

Funkcja preprocessing_maraton odpowiada za ostateczne przygotowanie danych przed walidacją schematu i modelowaniem. Kluczowe operacje to:

    Normalizacja typów danych: Konwersja kolumn kategorycznych na typ tekstowy oraz rygorystyczne wymuszenie typów numerycznych dla parametrów czasowych (eliminacja błędów formatowania CSV).
    Wyliczanie cech pochodnych (Feature Engineering):
        Tempo: Obliczenie średniego tempa biegu na podstawie czasu całkowitego i dystansu maratońskiego (42.195 km).
        Tempo_Stabilnosc: Wyznaczenie odchylenia standardowego z międzyczasów na poszczególnych odcinkach. Jest to kluczowa cecha określająca strategię biegu zawodnika (tzw. pacing), pozwalająca modelowi ML na odróżnienie biegaczy stabilnych od tych, którzy drastycznie zwalniają w drugiej połowie dystansu.

In [168]:
def preprocessing_maraton(df: pd.DataFrame) -> pd.DataFrame:
    # 1. Konwersja kolumn tekstowych (z użyciem pętli dla czystości)
    cols_to_str = ['Imię', 'Nazwisko', 'Miasto', 'Drużyna', 'Kraj']
    for col in cols_to_str:
        if col in df.columns:
            df[col] = df[col].astype('string')
    
    # 2. Bezpieczne obliczanie średniego tempa
    if 'Czas' in df.columns:
        # Wymuszamy typ numeryczny (coerce zamieni błędy na NaN)
        df['Czas'] = pd.to_numeric(df['Czas'], errors='coerce')
        df['Tempo'] = df['Czas'] / 42.195

    # 3. Obliczanie Stabilności na dostępnych odcinkach
    odcinki_tempo = ['5 km Tempo', '10 km Tempo', '15 km Tempo', '20 km Tempo']
    # Bierzemy tylko te kolumny, które faktycznie są w DataFrame
    existing_split_cols = [c for c in odcinki_tempo if c in df.columns]
    
    if existing_split_cols:
        # Wymuszamy typ numeryczny dla międzyczasów
        for col in existing_split_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        # Obliczamy odchylenie standardowe (miara stabilności)
        df['Tempo Stabilność'] = df[existing_split_cols].std(axis=1)

    return df

In [169]:
preprocessing_maraton(df_maraton23)
preprocessing_maraton(df_maraton24)


,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
1189,1248,19,KAROLINA,STAWARZ,OLEŚNICA,POL,KARUN,K,78,K20,...,4.690000,4263.000000,1377,4.860000,5744.000000,1288,4.936667,0.133250,6060.000000,143.618912
378,397,225,MILENA,BATOR,ZAWONIA,POL,OŚ RACING TEAM,K,20,K20,...,4.170000,3854.000000,426,4.380000,5208.000000,409,4.513333,0.144248,5501.000000,130.370897
215,227,380,OLHA,PUHACHOVA,KYIV,UKR,TS REGLE SZKLARSKA POREBA,K,7,K20,...,4.033333,3725.000000,228,4.216667,5025.000000,223,4.333333,0.124257,5321.000000,126.104989
20,23,387,NATALIJA,SEMENOVYCH,KIJÓW,UKR,Brak,K,1,K20,...,3.516667,3227.000000,22,3.716667,4362.000000,23,3.783333,0.135578,4616.000000,109.396848
397,416,403,AGATA,MAJ,Brak,NOR,Brak,K,22,K20,...,4.183333,3870.000000,447,4.376667,5226.000000,423,4.520000,0.138310,5525.000000,130.939685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10642,11540,85668,JANUSZ,DALEK,Brak,Brak,Brak,M,8326,M70,...,6.794848,6228.636364,11642,7.490303,8487.727273,11564,7.530303,0.521500,8977.818182,212.769716
10638,11508,86230,JERZY,CZYNIEWSKI,Brak,Brak,Brak,M,8307,M70,...,6.792424,6226.318182,11639,7.491818,8476.863636,11549,7.501818,0.516760,8963.909091,212.440078
9096,11482,9148,STANISŁAW,BEDNARSKI,STRZELIN,POL,Brak,M,8295,M80,...,6.790000,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440
11974,11482,10897,MARIAN,PAWLACZYK,Brak,Brak,Wkb Piast Wrocław,M,8295,M80,...,6.790000,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440


# Egzekucja Walidacji i Raportowanie Błędów

Poniższy blok kodu uruchamia proces weryfikacji danych dla obu roczników maratonu. Wykorzystanie trybu Lazy Validation pozwala na przechwycenie wszystkich niezgodności ze schematem jednocześnie. W przypadku wykrycia błędów (np. niewłaściwych typów danych lub naruszenia reguł biznesowych), generowany jest szczegółowy raport failure_cases, który wskazuje precyzyjną lokalizację problematycznych rekordów.

In [170]:
try:
    schema.validate(df_maraton23, lazy=True)
    schema.validate(df_maraton24, lazy=True)
except pa.errors.SchemaErrors as err:
    print("Walidacja nie powiodła się. Znalezione błędy:")
    # Pokazuje, które wiersze i kolumny nie przeszły walidacji
    print(err.failure_cases) 
    # Pokazuje oryginalne dane, które wywołały błąd
    #print(err.data)
    #print(err.data.head()) 

# Anonimizacja Danych Wrażliwych (Faker)

W celu ochrony prywatności uczestników maratonu oraz spełnienia standardów bezpieczeństwa danych (zgodność z RODO), zastosowano proces anonimizacji.
Dlaczego Faker?

    Eliminacja danych wrażliwych: Prawdziwe imiona i nazwiska biegaczy zostały zastąpione danymi syntetycznymi wygenerowanymi przez bibliotekę Faker.
    Zachowanie realizmu: W przeciwieństwie do zwykłego usuwania kolumn, Faker pozwala zachować strukturę danych (np. rozróżnienie płci na podstawie imion), co jest istotne dla modelu językowego (LLM) podczas generowania opisów klastrów.
    Bezpieczeństwo w chmurze: Dzięki temu procesowi, zbiory danych przechowywane w DigitalOcean Spaces oraz przesyłane do OpenAI API nie zawierają żadnych informacji pozwalających na identyfikację konkretnych osób fizycznych.

In [171]:
fake = Faker('pl_PL') 

def anonymize_marathon_data(df: pd.DataFrame) -> pd.DataFrame:
    # Tworzymy kopię, żeby nie modyfikować oryginalnych danych
    df_anonymized = df.copy()
    
    # 1. Anonimizacja danych osobowych
    if 'Imię' in df.columns:
        df_anonymized['Imię'] = [fake.first_name() for _ in range(len(df))]
        
    if 'Nazwisko' in df.columns:
        df_anonymized['Nazwisko'] = [fake.last_name() for _ in range(len(df))]

    # 2. Anonimizacja danych geograficznych/organizacyjnych
    if 'Miasto' in df.columns:
        df_anonymized['Miasto'] = [fake.city() for _ in range(len(df))]
        
    if 'Drużyna' in df.columns:
        # Można użyć nazw firm jako nazw drużyn
        df_anonymized['Drużyna'] = [fake.company() for _ in range(len(df))]

    # Jeśli zdecydujesz się anonimizować kraj:
    # if 'Kraj' in df.columns:
    #     df_anonymized['Kraj'] = [fake.country_code() for _ in range(len(df))]

    return df_anonymized

In [172]:
df_maraton23 = anonymize_marathon_data(df_maraton23)
df_maraton23

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
6551,7207,100,Maurycy,Śmieszek,Starachowice,POL,Fundacja Dąbrówka-Krysik Sp.k.,K,1748,K20,...,6.040000,5559.0,6750,6.613333,7936.0,7156,7.923333,0.928707,8376.0,198.506932
1029,1066,117,Tola,Hinca,Pruszcz Gdański,POL,Tyka-Maszkiewicz s.c.,K,73,K20,...,4.606667,4278.0,1306,4.816667,5789.0,1106,5.036667,0.175760,6047.0,143.310819
8138,8939,119,Kamila,Chwedoruk,Siemianowice Śląskie,POL,Zackiewicz-Nestorowicz s.c.,K,2607,K20,...,8.570000,7747.0,8934,9.476667,10995.0,8939,10.826667,1.308911,11581.0,274.463799
6523,7177,121,Marcin,Tekieli,Bydgoszcz,POL,Gładka-Guźniczak Sp.j.,K,1732,K20,...,6.210000,5708.0,7154,6.636667,7940.0,7163,7.440000,0.587178,8351.0,197.914445
8215,7292,143,Melania,Melka,Ostrów Wielkopolski,Brak,Samoraj-Masłoń s.c.,K,1791,K20,...,6.286667,5770.5,7293,6.751667,8019.0,7272,7.495000,0.593762,8433.5,199.869653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6448,7092,8535,Dariusz,Kosiec,Dzierżoniów,POL,Gabinety Malewicz,M,5398,M70,...,6.060000,5559.0,6750,6.443333,7874.0,7057,7.716667,0.792869,8293.0,196.539874
7229,7971,8703,Fabian,Drobniak,Stalowa Wola,POL,Dziuda Sp.j.,M,5855,M70,...,7.033333,6168.0,8118,7.133333,8502.0,7971,7.780000,0.567578,8940.0,211.873445
2188,2314,8965,Daniel,Lewko,Zawiercie,POL,Fundacja Kurtyka Sp. z o.o. Sp.k.,M,2092,M70,...,4.856667,4407.0,1761,5.226667,6211.0,2194,6.013333,0.613680,6556.0,155.373859
8136,8937,2797,Angelika,Wardzała,Kielce,POL,Gawliczek-Szóstak Sp.j.,M,6331,M80,...,8.766667,7773.0,8936,9.013333,10810.0,8937,10.123333,0.831289,11370.0,269.463207


In [173]:
df_maraton24 = anonymize_marathon_data(df_maraton24)
df_maraton24

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,10 km Tempo,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo
1189,1248,19,Anita,Drygas,Stalowa Wola,POL,Grupa Drewnik S.A.,K,78,K20,...,4.690000,4263.000000,1377,4.860000,5744.000000,1288,4.936667,0.133250,6060.000000,143.618912
378,397,225,Hubert,Forysiak,Żory,POL,Grupa Wiraszka Sp. z o.o. Sp.k.,K,20,K20,...,4.170000,3854.000000,426,4.380000,5208.000000,409,4.513333,0.144248,5501.000000,130.370897
215,227,380,Kamila,Strzyż,Oleśnica,UKR,Spółdzielnia Maksym Sp. z o.o.,K,7,K20,...,4.033333,3725.000000,228,4.216667,5025.000000,223,4.333333,0.124257,5321.000000,126.104989
20,23,387,Grzegorz,Suchojad,Słupsk,UKR,Symonowicz-Idec Sp. z o.o.,K,1,K20,...,3.516667,3227.000000,22,3.716667,4362.000000,23,3.783333,0.135578,4616.000000,109.396848
397,416,403,Błażej,Simon,Jaworzno,NOR,Maleszka Sp.j.,K,22,K20,...,4.183333,3870.000000,447,4.376667,5226.000000,423,4.520000,0.138310,5525.000000,130.939685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10642,11540,85668,Iwo,Gregorek,Reda,Brak,Brygoła Sp. z o.o. Sp.k.,M,8326,M70,...,6.794848,6228.636364,11642,7.490303,8487.727273,11564,7.530303,0.521500,8977.818182,212.769716
10638,11508,86230,Dominik,Brudny,Opole,Brak,Stowarzyszenie Wawszczak,M,8307,M70,...,6.792424,6226.318182,11639,7.491818,8476.863636,11549,7.501818,0.516760,8963.909091,212.440078
9096,11482,9148,Jan,Klos,Płońsk,POL,FPUH Hynek,M,8295,M80,...,6.790000,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440
11974,11482,10897,Jędrzej,Krzyszczak,Dąbrowa Górnicza,Brak,Stawowczyk s.c.,M,8295,M80,...,6.790000,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440


In [174]:
df_maraton23['Rok_maratonu'] = 2023
df_maraton24['Rok_maratonu'] = 2024
df_maraton = pd.concat([df_maraton23, df_maraton24], ignore_index=True)
df_maraton

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,15 km Czas,15 km Miejsce Open,15 km Tempo,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Rok_maratonu
0,7207,100,Maurycy,Śmieszek,Starachowice,POL,Fundacja Dąbrówka-Krysik Sp.k.,K,1748,K20,...,5559.000000,6750,6.613333,7936.000000,7156,7.923333,0.928707,8376.000000,198.506932,2023
1,1066,117,Tola,Hinca,Pruszcz Gdański,POL,Tyka-Maszkiewicz s.c.,K,73,K20,...,4278.000000,1306,4.816667,5789.000000,1106,5.036667,0.175760,6047.000000,143.310819,2023
2,8939,119,Kamila,Chwedoruk,Siemianowice Śląskie,POL,Zackiewicz-Nestorowicz s.c.,K,2607,K20,...,7747.000000,8934,9.476667,10995.000000,8939,10.826667,1.308911,11581.000000,274.463799,2023
3,7177,121,Marcin,Tekieli,Bydgoszcz,POL,Gładka-Guźniczak Sp.j.,K,1732,K20,...,5708.000000,7154,6.636667,7940.000000,7163,7.440000,0.587178,8351.000000,197.914445,2023
4,7292,143,Melania,Melka,Ostrów Wielkopolski,Brak,Samoraj-Masłoń s.c.,K,1791,K20,...,5770.500000,7293,6.751667,8019.000000,7272,7.495000,0.593762,8433.500000,199.869653,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21952,11540,85668,Iwo,Gregorek,Reda,Brak,Brygoła Sp. z o.o. Sp.k.,M,8326,M70,...,6228.636364,11642,7.490303,8487.727273,11564,7.530303,0.521500,8977.818182,212.769716,2024
21953,11508,86230,Dominik,Brudny,Opole,Brak,Stowarzyszenie Wawszczak,M,8307,M70,...,6226.318182,11639,7.491818,8476.863636,11549,7.501818,0.516760,8963.909091,212.440078,2024
21954,11482,9148,Jan,Klos,Płońsk,POL,FPUH Hynek,M,8295,M80,...,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440,2024
21955,11482,10897,Jędrzej,Krzyszczak,Dąbrowa Górnicza,Brak,Stawowczyk s.c.,M,8295,M80,...,6224.000000,11630,7.493333,8466.000000,11525,7.473333,0.512326,8950.000000,212.110440,2024


# Inżynieria Cech i Przygotowanie Danych (Feature Engineering)

Funkcja prepare_marathon_data pełni kluczową rolę w transformacji surowych danych maratońskich na format wejściowy dla algorytmów uczenia maszynowego. Proces ten obejmuje trzy kluczowe etapy:

    Ekstrakcja Cech Demograficznych (Parsing):
        Przetworzenie kolumny „Kategoria wiekowa” w celu wyodrębnienia numerycznej grupy wiekowej oraz zamiana płci na format One-Hot Encoding (Plec_K, Plec_M). Pozwala to modelowi na matematyczną interpretację różnic biologicznych i wiekowych między biegaczami.
    Selekcja Cech (Feature Selection):
        Wybór unikalnego zestawu zmiennych łączących dynamikę biegu (międzyczasy i czas końcowy) z danymi demograficznymi. Dzięki temu klastrowanie odbywa się w wielowymiarowej przestrzeni uwzględniającej zarówno tempo, jak i profil biegacza.
    Standaryzacja (StandardScaler):
        Kluczowy krok dla algorytmów opartych na odległościach (np. K-Means). Standaryzacja sprowadza cechy o różnych jednostkach (sekundy vs wiek) do wspólnej skali o średniej 0 i odchyleniu 1, co zapobiega dominacji modelu przez zmienne o większych wartościach liczbowych.

In [175]:
def prepare_marathon_data(df):
    # 1. Wyodrębnienie płci i wieku z kolumny 'Kategoria wiekowa' (np. M30 -> M, 30)
    # Korzystamy z [Series.str.extract](https://pandas.pydata.org)
    df['Plec_K'] = df['Płeć'].apply(lambda x: 1 if 'K' in str(x).upper() else 0)
    df['Plec_M'] = df['Płeć'].apply(lambda x: 1 if 'M' in str(x).upper() else 0)
    
    # Wyciągamy same cyfry dla grupy wiekowej
    df['Grupa_Wiekowa'] = df['Kategoria wiekowa'].str.extract('(\d+)').fillna(0).astype(int)

    # 2. Wybór kolumn do uczenia (X)
    # Tu muszą być wszystkie zmienne: czas + wiek + płeć
    features = ['5 km Czas','10 km Czas','15 km Czas','20 km Czas','Czas' ,'Grupa_Wiekowa', 'Plec_K', 'Plec_M']
    X = df[features]

    # 3. Skalowanie - kluczowy krok dla algorytmów dystansowych (K-Means, MeanShift)
    # Używamy [StandardScaler](https://scikit-learn.org)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    return X_scaled, df

x,dfx = prepare_marathon_data(df_maraton)

In [176]:
dfx.sample(10)

,Miejsce,Numer startowy,Imię,Nazwisko,Miasto,Kraj,Drużyna,Płeć,Płeć Miejsce,Kategoria wiekowa,...,20 km Czas,20 km Miejsce Open,20 km Tempo,Tempo Stabilność,Czas,Tempo,Rok_maratonu,Plec_K,Plec_M,Grupa_Wiekowa
21640,8232,4554,Jakub,Gromala,Zambrów,Brak,Błaszyk Sp. z o.o. Sp.k.,M,6511,M60,...,7392.000000,8140,7.085000,0.679850,7856.500000,186.195047,2024,0,1,60
5211,8103,6930,Albert,Surdyk,Brodnica,POL,Gabinety Kwinta-Świętoń i syn s.c.,M,5920,M30,...,8507.000000,7979,9.393333,1.664335,9046.000000,214.385591,2023,0,1,30
15055,473,1575,Radosław,Prejs,Jawor,POL,FPUH Myga-Ziemann Sp.k.,M,450,M30,...,5280.000000,478,4.496667,0.100922,5585.000000,132.361654,2024,0,1,30
8685,7297,2291,Ewelina,Kaczka,Sieradz,POL,Haremza-Kazana Sp. z o.o.,M,5504,M60,...,8028.000000,7289,7.763333,0.878901,8436.000000,199.928902,2023,0,1,60
159,8264,2905,Norbert,Skalik,Białogard,POL,Stowarzyszenie Mosiołek,K,2268,K20,...,8783.000000,8292,8.370000,0.714798,9205.000000,218.153810,2023,1,0,20
19622,6564,8486,Krzysztof,Dacko,Dębica,Brak,Fundacja Porzucek-Winnik i syn s.c.,M,5455,M40,...,7063.000000,6651,6.088333,0.172592,7461.500000,176.833748,2024,0,1,40
8675,7694,1924,Maurycy,Staciwa,Koło,POL,Spółdzielnia Gierasimiuk,M,5710,M60,...,8284.000000,7692,7.500000,0.438643,8697.000000,206.114469,2023,0,1,60
4088,1548,1949,Olgierd,Dudczak,Bielawa,POL,Spółdzielnia Betka Sp.j.,M,1432,M30,...,5970.000000,1537,5.500000,0.370970,6269.000000,148.572106,2023,0,1,30
12538,7513,85319,Marianna,Rup,Jelenia Góra,Brak,Siemek S.A.,K,1476,K40,...,7254.593301,7504,6.503014,0.366715,7694.870813,182.364517,2024,1,0,40
8073,648,2182,Sara,Węgier,Kwidzyn,POL,PPUH Bazydło,M,606,M50,...,5511.000000,657,4.876667,0.231858,5770.000000,136.746060,2023,0,1,50


# Upload to digital ocean

Gotowy zbiór treningowy wysyłamy do serwera digital ocean

In [177]:
def upload_df_to_digital_ocean(df,access_key,access_secret_key, nazwa_space,plik_csv,region = 'fra1',end_url='https://fra1.digitaloceanspaces.com'):
    
    session = boto3.session.Session()
    client = session.client(
        's3',
        region_name=region.strip(),
        endpoint_url=end_url.strip(), # Używamy endpointu bazowego do komunikacji API
        aws_access_key_id=access_key.strip(),
        aws_secret_access_key=access_secret_key.strip()
    )
    # 1. Przygotowanie bufora tekstowego (wirtualny plik w RAM)
    csv_buffer = io.StringIO()
    
    # 2. Zapisanie DataFrame do bufora (z Twoim separatorem i kodowaniem)
    df.to_csv(csv_buffer, sep=';', encoding='utf-8-sig', index=False)
    
    try:
        # 3. Wysłanie zawartości bufora do DigitalOcean
        client.put_object(
            Bucket=nazwa_space,
            Key=plik_csv,
            Body=csv_buffer.getvalue(),
            ContentType='text/csv'
        )
        print(f"Plik {plik_csv} wysłany bezpośrednio do {nazwa_space}")
    except Exception as e:
        print(f"Błąd wysyłki: {e}")

In [178]:
df_maraton.to_csv('halfmarathon_wroclaw_final.csv', sep=';', index=False, encoding='utf-8')
do_access_key, do_secret_key, do_space_input, do_region, do_end_url = handle_digital_ocean_keys()
upload_df_to_digital_ocean(df_maraton,do_access_key, do_secret_key, do_space_input, 'halfmarathon_wroclaw_final.csv',do_region, do_end_url)

Plik halfmarathon_wroclaw_final.csv wysłany bezpośrednio do maraton


# Trenowanie i budowa modelu treningowego

Najważniejsza funkcja do budowy modelu treningowego. Można w niej wybrać ilość grup klastrów, na które funkcja będzie starać się podzielić. Można też wybrać, czy zastosować normalizację i transformację. 
1) Z-Score Normalization – aby sprowadzić cechy o różnych jednostkach (wiek, sekundy, punkty stabilności) do wspólnej skali, zapewniając każdemu z nich równy wpływ na ostateczny wynik grupowania.
2) Quantile Transformation – w celu zniwelowania asymetrii rozkładów (szczególnie w czasach i tempie) i zbliżenia ich do rozkładu normalnego, co poprawia stabilność algorytmów klastrujących.

Wybór optymalnego modelu klastrowania został oparty na analizie porównawczej następujących wskaźników:

    1) Precision: Miara wiarygodności przypisania – minimalizuje ryzyko błędnego zakwalifikowania biegacza do niewłaściwej grupy profilowej.
    2) Recall: Miara kompletności – gwarantuje, że model potrafi zidentyfikować większość reprezentantów danej charakterystyki biegowej.
    3) F1-Score: Kluczowy wskaźnik optymalizujący kompromis między precyzją a czułością, zapewniający stabilność modelu.
    4) Min_Pct: Parametr kontrolny dbający o odpowiednią liczebność grup, zapobiegający nadmiernej fragmentacji danych.
    5) Homogeneity & Completeness: Pozwalają ocenić, na ile klastry odzwierciedlają naturalne podziały (np. płeć, wiek), mimo że model nie widział tych etykiet podczas treningu.
    6) Cluster Size (Min/Max/Pct): Parametry te gwarantują użyteczność biznesową modelu – eliminujemy rozwiązania, które tworzą zbyt małe (nieistotne statystycznie) lub zbyt duże (mało precyzyjne) grupy.
    7) n_clusters: Kluczowy parametr kontrolujący ziarnistość podziału społeczności maratońskiej.
    

In [179]:
def build_model(MODEL_ID,DATA,y_true,num_clusters=8, normalize=False, transform=False):
    normalize_method ='zscore'
    transformation_method='quantile'
    dane_numeryczne = DATA.select_dtypes(include=['number'])
    s = setup(data = dane_numeryczne,         
        normalize=normalize, normalize_method=normalize_method,
        transformation=transform, transformation_method=transformation_method,
        session_id=123, html=False, verbose=False,memory=False,low_variance_threshold=0.01)
    
    model = create_model(MODEL_ID, num_clusters, verbose=False)

     # Pobieranie etykiet przypisanych przez model
    labels = assign_model(model)['Cluster'] # PyCaret dodaje kolumnę 'Cluster'
    
    # !!! WAŻNE !!!
    # Konwertujemy etykiety klastrów na format liczbowy (0, 1, 2...), bo sklearn tego wymaga
    labels_numeric = labels.str.replace('Cluster ', '').astype(int)
    
    # 1. Liczymy statystyki liczebności (kod z poprzedniej odpowiedzi)
    counts = pd.Series(labels).value_counts()
    count_min_pct = (counts.min() / len(labels)) * 100 
    
    # 2. Liczymy metryki KLASYFIKACJI (nowość!)
    # Scikit-learn potrzebuje y_true i labels jako numeryczne
    # Musisz najpierw upewnić się, że y_true też jest numeryczne (np. 1, 2, 3...)
    # (Zakładam, że masz już LabelEncoder dla y_true wczesniej w skrypcie)
    
    # Ponieważ te metryki działają na klasyfikacji wieloklasowej,
    # używamy parametru 'average="weighted"' lub "macro"
    
    le = LabelEncoder()
    y_true_numeric = le.fit_transform(y_true)

    # accuracy = accuracy_score(y_true_numeric, labels_numeric) # Accuracy jest mylace w klastrowaniu
    precision = precision_score(y_true_numeric, labels_numeric, average='weighted', zero_division=0)
    recall = recall_score(y_true_numeric, labels_numeric, average='weighted', zero_division=0)
    f1 = f1_score(y_true_numeric, labels_numeric, average='weighted', zero_division=0)
    
    # AUC jest trudne do policzenia dla klastrowania/wieloklasowego, pomijamy je na razie
    
    # 3. Dodajemy nowe kolumny do tabeli metryk
    metrics = pull() # Standardowe metryki PyCaret
    metrics['Min_Pct'] = round(count_min_pct, 2)
    metrics['Precision'] = round(precision, 3)
    metrics['Recall'] = round(recall, 3)
    metrics['F1_Score'] = round(f1, 3)
    # 2. Liczymy statystyki liczebności
    counts = pd.Series(labels).value_counts()
    count_min = counts.min()
    count_max = counts.max()
    count_min_pct = (count_min / len(labels)) * 100 # Procentowo
    metrics = pull()

    # Ręczne liczenie brakujących metryk (tych, co miałeś na 0)
    h_score = homogeneity_score(y_true, labels)
    c_score = completeness_score(y_true, labels)

    # Dodanie naszych metryk do tabeli wyników
    metrics['Homogeneity'] = h_score
    metrics['Completeness'] = c_score
    metrics['Min_Cluster_Size'] = count_min
    metrics['Min_Cluster_Pct'] = round(count_min_pct, 2)
    metrics['Max_Cluster_Size'] = count_max
    metrics['n_clusters'] = len(counts)

    name = f"{MODEL_ID.upper()}_k{num_clusters}_n_{normalize}_t_{transform}"
    metrics.index = [name]
    metrics.insert(0,"model",name)

    return model, metrics


# Porównanie i selekcja modeli klastrujących

W projekcie przetestowano cztery fundamentalnie różne podejścia do segmentacji danych, aby znaleźć najbardziej stabilny podział społeczności maratońskiej:

    K-Means (Podział oparty na centrach):
        Dlaczego: Klasyk i standard w klastrowaniu. Szybki i efektywny, gdy grupy mają kształt zbliżony do kulistego. Idealny do wyznaczenia wyraźnych centrów profili biegaczy (np. „typowy amator”).
    BIRCH (Podejście hierarchiczne i skalowalne):
        Dlaczego: Świetnie radzi sobie z dużymi zbiorami danych (ponad 21 tys. rekordów). Buduje drzewo cech, co pozwala na szybszą analizę bez konieczności trzymania wszystkich danych w pamięci RAM. Jest mniej wrażliwy na szum niż K-Means.
    Affinity Propagation (AP - Przekazywanie podobieństwa):
        Dlaczego: W przeciwieństwie do K-Means, ten model nie wymaga wcześniejszego podania liczby klastrów. Sam wybiera „reprezentantów” (egzemplarze) grup, co pozwala odkryć naturalną strukturę danych, której mogliśmy nie przewidzieć.
    MeanShift (Przesunięcie ku gęstości):
        Dlaczego: Algorytm poszukuje obszarów o największym zagęszczeniu biegaczy w przestrzeni cech. Jest bardzo dobry w ignorowaniu outlierów (np. osób o absurdalnych czasach) i skupianiu się na realnych trendach wewnątrz grup.


In [180]:
results_storage = {}
all_metrics_list = [] # Lista pomocnicza do zbiorczej tabeli

# Przykładowe dane (użyj swojego df_maraton lub próbki)
# df_data = df_maraton.sample(2000) 
df_data = dfx.sample(2000) # Uważaj na MemoryError dla hclust/ap na pełnych danych!
y_true_col = df_data['Kategoria wiekowa'] 

# Lista eksperymentów do wykonania (model, liczba klastrów,normalize, transform)
experiments = [
    ('kmeans', 5,False,False),
    ('kmeans', 10,False,False),
    ('birch', 5,False,False),
    ('birch', 10,False,False),
    ('ap', 5,False,False),
    ('ap', 10,False,False),
    ('meanshift', 5,False,False),
    ('meanshift', 10,False,False),
    ('kmeans', 5,True,False),
    ('kmeans', 10,True,False),
    ('birch', 5,True,False),
    ('birch', 10,True,False),
    ('ap', 5,True,False),
    ('ap', 10,True,False),
     ('meanshift', 5,True,False),
    ('meanshift', 10,True,False),
    ('kmeans', 5,False,True),
    ('kmeans', 10,False,True),
    ('birch', 5,False,True),
    ('birch', 10,False,True),
    ('ap', 5,False,True),
    ('ap', 10,False,True),
    ('meanshift', 5,False,True),
    ('meanshift', 10,False,True),
    ('kmeans', 5,True,True),
    ('kmeans', 10,True,True),
    ('birch', 5,True,True),
    ('birch', 10,True,True),
    ('ap', 5,True,True),
    ('ap', 10,True,True),
    ('meanshift', 5,True,True),
    ('meanshift', 10,True,True),

]

for model_type, k_value,normalize,transform in experiments:
    # Generujemy unikalną nazwę dla eksperymentu
    exp_name = f"{model_type.upper()}_k{k_value}_n_{normalize}_t_{transform}"
    
    print(f"Uruchamiam eksperyment: {exp_name}")
    
    # Trenujemy model i odbieramy 2 wartości
    model, metrics_df = build_model(model_type, df_data, y_true_col,num_clusters=k_value, normalize=normalize, transform=transform)
    
    # Przechowujemy pełny wynik w słowniku
    results_storage[exp_name] = (model, metrics_df)
    
    # Dodajemy metryki do listy do późniejszego porównania
    all_metrics_list.append(metrics_df)




Uruchamiam eksperyment: KMEANS_k5_n_False_t_False
Uruchamiam eksperyment: KMEANS_k10_n_False_t_False
Uruchamiam eksperyment: BIRCH_k5_n_False_t_False
Uruchamiam eksperyment: BIRCH_k10_n_False_t_False
Uruchamiam eksperyment: AP_k5_n_False_t_False
Uruchamiam eksperyment: AP_k10_n_False_t_False
Uruchamiam eksperyment: MEANSHIFT_k5_n_False_t_False
Uruchamiam eksperyment: MEANSHIFT_k10_n_False_t_False
Uruchamiam eksperyment: KMEANS_k5_n_True_t_False
Uruchamiam eksperyment: KMEANS_k10_n_True_t_False
Uruchamiam eksperyment: BIRCH_k5_n_True_t_False
Uruchamiam eksperyment: BIRCH_k10_n_True_t_False
Uruchamiam eksperyment: AP_k5_n_True_t_False
Uruchamiam eksperyment: AP_k10_n_True_t_False
Uruchamiam eksperyment: MEANSHIFT_k5_n_True_t_False
Uruchamiam eksperyment: MEANSHIFT_k10_n_True_t_False
Uruchamiam eksperyment: KMEANS_k5_n_False_t_True
Uruchamiam eksperyment: KMEANS_k10_n_False_t_True
Uruchamiam eksperyment: BIRCH_k5_n_False_t_True
Uruchamiam eksperyment: BIRCH_k10_n_False_t_True
Uruchamiam e

# Analiza Porównawcza i Wybór Modelu

W celu wyłonienia optymalnego algorytmu, wyniki wszystkich modeli zostały zestawione w tabeli zbiorczej. Kluczowymi wskaźnikami, na których oparto decyzję o wyborze modelu produkcyjnego, są:

    Silhouette Score (Podświetlony):
        Co oznacza: Mierzy, jak blisko swojego klastra znajduje się każdy punkt w porównaniu do klastrów sąsiednich.
        Interpretacja: Wysoka wartość Silhouette wskazuje, że klastry są dobrze odseparowane i nie nakładają się na siebie. Jest to kluczowe dla LLM – wyraźne różnice w danych wejściowych pozwalają na wygenerowanie unikalnych i trafnych opisów dla każdej grupy.
    Homogeneity (Podświetlony):
        Co oznacza: Sprawdza, czy klastry są spójne pod kątem cech kategorycznych (np. płeć, kategorie wiekowe).
        Interpretacja: Wysoka jednorodność oznacza, że podział wygenerowany przez algorytm "pokrywa się" z naturalnymi podziałami społecznymi biegaczy. Dzięki temu nazwy klastrów (np. "Szybkie Kobiety") są osadzone w rzeczywistości, a nie są jedynie abstrakcyjnym zbiorem liczb.

Wniosek: Model z najwyższymi wartościami w tych polach zapewnia najlepszy balans między matematyczną poprawnością a możliwością interpretacji wyników przez człowieka.

In [181]:
# --- Analiza wyników ---

# 1. Zbiorcza tabela metryk (do szybkiego porównania)
df_comparison = pd.concat(all_metrics_list, ignore_index=True)
print("\n--- Tabela Porównawcza Metryk ---")


# Funkcja pomocnicza do "inteligentnego" kolorowania rozmiaru klastra
def color_min_pct(val):
    if 2.0 <= val <= 15.0:
        return 'background-color: #c7e9c0; color: black' # Jasnozielony - idealnie
    elif val < 2.0:
        return 'background-color: #ff9999; color: black' # Czerwony - za małe!
    return '' # Reszta neutralna

# Budujemy Stylera
df_final_styled = df_comparison.style.applymap(
    color_min_pct, subset=['Min_Cluster_Pct']
).background_gradient(
    subset=['Homogeneity', 'Silhouette'], 
    cmap='YlGn' # Żółto-zielony gradient dla jakości
).background_gradient(
    subset=['Max_Cluster_Size'], 
    cmap='YlOrRd' # Żółto-pomarańczowy-czerwony (im większy klaster, tym bardziej ostrzegawczo)
).applymap(
    lambda x: 'background-color: #74c476; font-weight: bold' if 5 <= x <= 12 else '',
    subset=['n_clusters'] # Zielony dla Twojej liczby opisów w JSON
).format({
    'Min_Cluster_Pct': '{:.2f}%',
    'Homogeneity': '{:.3f}',
    'Silhouette': '{:.3f}'
})

# Przygotowanie ostatecznej, stylowej tabeli
df_final_styled = df_comparison.style \
    .background_gradient(subset=['Homogeneity', 'Silhouette'], cmap='YlGn') \
    .applymap(color_min_pct, subset=['Min_Cluster_Pct']) \
    .hide() \
    .set_properties(subset=['model'], **{'font-weight': 'bold', 'text-align': 'left'}) \
    .highlight_max(subset=['Homogeneity'], props='font-weight: bold; color: white; background-color: #2e7d32;') \
    .format(precision=3)


# Wyświetlenie w Notebooku
show(df_final_styled, classes="display nowrap cell-border") # itables pozwala klikać w nagłówki!



--- Tabela Porównawcza Metryk ---


model,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness,Min_Pct,Precision,Recall,F1_Score,Min_Cluster_Size,Min_Cluster_Pct,Max_Cluster_Size,n_clusters
KMEANS_k5_n_False_t_False,0.416,4827.751,0.714,0.028,0,0.043,2.750,0.025,0.048,0.025,55,2.750,707,5
KMEANS_k10_n_False_t_False,0.292,3641.670,1.109,0.052,0,0.050,1.650,0.130,0.068,0.076,33,1.650,367,10
BIRCH_k5_n_False_t_False,0.379,4382.703,0.732,0.027,0,0.041,2.750,0.035,0.072,0.040,55,2.750,661,5
BIRCH_k10_n_False_t_False,0.255,3279.009,1.150,0.076,0,0.076,1.500,0.079,0.050,0.047,30,1.500,516,10
AP_k5_n_False_t_False,0.274,2361.987,1.038,0.319,0,0.172,0.150,0.203,0.031,0.051,3,0.150,117,54
AP_k10_n_False_t_False,0.274,2361.987,1.038,0.319,0,0.172,0.150,0.203,0.031,0.051,3,0.150,117,54
MEANSHIFT_k5_n_False_t_False,0.481,1951.523,0.666,0.009,0,0.032,0.150,0.030,0.074,0.030,3,0.150,1654,4
MEANSHIFT_k10_n_False_t_False,0.481,1951.523,0.666,0.009,0,0.032,0.150,0.030,0.074,0.030,3,0.150,1654,4
KMEANS_k5_n_True_t_False,0.272,813.198,1.402,0.250,0,0.334,11.750,0.074,0.094,0.082,235,11.750,611,5
KMEANS_k10_n_True_t_False,0.223,605.597,1.419,0.286,0,0.264,2.750,0.243,0.161,0.184,55,2.750,364,10


# Selekcja Modelu Produkcyjnego (Model Selection Logic)

Proces wyboru ostatecznego modelu nie opiera się wyłącznie na najwyższych metrykach, ale przede wszystkim na kryteriach stabilności operacyjnej. Zastosowano wielostopniowy filtr bezpieczeństwa:

    Kompatybilność API: Ograniczono wybór do modeli K-Means i BIRCH, które natywnie wspierają metodę .predict(). Jest to niezbędne do poprawnego działania aplikacji w trybie "na żywo" na DigitalOcean.
    Reprezentatywność grup (Min_Cluster_Pct >= 2%): Wyeliminowano modele tworzące zbyt małe klastry (tzw. outliery), które nie niosą wartości statystycznej i mogłyby mylić model językowy LLM.
    Zdolność do uogólniania (Max_Cluster_Size < 80%): Odrzucono modele, które "idą na łatwiznę", wrzucając większość biegaczy do jednego ogromnego worka. Dążymy do realnego podziału społeczności.
    Złożoność (n_clusters <= 15): Ustalono limit grup, aby zachować czytelność raportu i zmieścić się w limitach kontekstu (Context Window) API OpenAI.

Ostateczny wybór: Spośród modeli spełniających powyższe rygory, wybrano ten o najwyższym wskaźniku Homogeneity, co gwarantuje, że podział matematyczny najlepiej pokrywa się z rzeczywistymi cechami biegaczy.

In [182]:
# Łączymy wyniki
df_results = pd.concat(all_metrics_list)

# FILTRUJEMY: 
# 1. Tylko modele z metodą predict (KMEANS, BIRCH)
# 2. Minimum 2% danych w najmniejszym klastrze (żadnych 'samotników')
# 3. Maksymalnie 15 klastrów (żeby zmieściły się w JSON)
# 4. Największy klaster nie może mieć więcej niż 80% danych (żeby model faktycznie dzielił)

df_safe = df_results[
    (df_results['model'].str.contains('KMEANS|BIRCH')) & 
    (df_results['Min_Cluster_Pct'] >= 2.0) & 
    (df_results['n_clusters'] <= 15) &
    (df_results['Max_Cluster_Size'] < (len(df_data) * 0.8))
]

# Z tych bezpiecznych wybieramy ten z najwyższym Homogeneity
if not df_safe.empty:
    best_safe_model_name = df_safe.sort_values(by='Homogeneity', ascending=False).iloc[0]['model']
    print(f"Najlepszy model do APLIKACJI: {best_safe_model_name}")
else:
    print("Brak modelu spełniającego kryteria bezpieczeństwa. Spróbuj zwiększyć k w K-Means lub włączyć normalizację.")

Najlepszy model do APLIKACJI: BIRCH_k10_n_False_t_True


# Zapis wybranego modelu

Na koniec analizy zapisujemy model lokalnie i w digital ocean.

In [183]:
# 1. Pobieramy nazwę z tabeli
best_name = str(best_safe_model_name).strip().lower()

# 2. Szukamy pasującego klucza w słowniku
actual_key = None
for key in results_storage.keys():
    if key.strip().lower() == best_name:
        actual_key = key
        break

if actual_key:
    # Wyciągamy model (pierwszy element pary)
    best_model_object = results_storage[actual_key][0]
    
    # 3. Zapisujemy
    from pycaret.clustering import save_model
    nazwa_pliku = "model_maraton"
    save_model(best_model_object, nazwa_pliku)
    print(f"Sukces! Model zapisany lokalnie pod kluczem: {actual_key}")
    #print(f"Zawartość modelu: {best_model_object}")
    do_access_key, do_secret_key, do_space_input, do_region, do_end_url = handle_digital_ocean_keys()
    upload_model_to_digital_ocean(nazwa_pliku + ".pkl",do_access_key, do_secret_key, do_space_input,None,do_region, do_end_url)
else:
    print(f"Błąd: Nie znaleziono klucza '{best_name}' w słowniku.")
    print(f"Dostępne klucze to: {list(results_storage.keys())[:5]}...")

Transformation Pipeline and Model Successfully Saved
Sukces! Model zapisany lokalnie pod kluczem: BIRCH_k10_n_False_t_True
Model model_maraton.pkl został wysłany do Space: maraton jako model_maraton.pkl
